# Imports

In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1_period/model/')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
import random
from scipy import stats
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score

from pipeline_config import *
from reversed_hgru_data_preprocess.preprocess_config import *

import os

#from Prices.prices_over_time import *

In [3]:
torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

In [4]:
Year

2020

In [5]:
hgru_test_preds_path = '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model/test_predictions.pickle'

# Read Results Data

In [6]:
with open(test_predictions_path_with_hgru, 'rb') as f:
    prediction_dic = pickle.load(f)

with open(hgru_test_preds_path, 'rb') as f:
    hgru_prediction_dic = pickle.load(f)

with open(HRNNpath, 'rb') as f:
    raw_dataset_dict = pickle.load(f)
    
with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

In [7]:
def create_test_dataframe(raw_dataset_dict: dict):
    test_dict = {}
    for key in raw_dataset_dict.keys():
        df = raw_dataset_dict[key][['Category', 'Date', 'Year', 'Indent', 'Inflation t+1']]
        df.dropna(inplace=True)
        df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
        target_df = df[df['Year'] > Year]
        test_dict[key] = target_df
    return test_dict

test_dict = create_test_dataframe(raw_dataset_dict)


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_48315/1314998454.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_48315/1314998454.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_48315/1314998454.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [8]:
def get_df_with_predictions(prediction_dic: dict, dict_of_categories_df:dict) -> dict:
    all_data_dict = {}
    for key in list(prediction_dic.keys()):
        predictions = prediction_dic[key]
        prediction_df =  pd.DataFrame(predictions.flatten().detach().numpy())
        prediction_df.rename(columns = {0: 'Prediction'}, inplace=True)
        dict_of_categories_df[key] = dict_of_categories_df[key].reset_index(drop=True)
        all_data_dict[key] = pd.concat([dict_of_categories_df[key], prediction_df], axis=1)
    return all_data_dict

all_data_test_dict=get_df_with_predictions(prediction_dic, test_dict)

In [9]:
def get_df_with_predictions_with_hgru(prediction_dic: dict, hgru_pred_dic: dict, dict_of_categories_df:dict) -> dict:
    bidirect_hgru_dict = {}
    for key in list(prediction_dic.keys()):
        predictions = prediction_dic[key]
        prediction_df =  pd.DataFrame(predictions.flatten().detach().numpy())
        hgru_predictions = hgru_pred_dic[key]
        hgru_prediction_df =  pd.DataFrame(hgru_predictions.flatten().detach().numpy())

        prediction_df.rename(columns = {0: 'Bidirectional_Prediction'}, inplace=True)
        hgru_prediction_df.rename(columns = {0: 'HGRU_Prediction'}, inplace=True)

        dict_of_categories_df[key] = dict_of_categories_df[key].reset_index(drop=True)

        bidirect_hgru_dict[key] = pd.concat([dict_of_categories_df[key], prediction_df, hgru_prediction_df], axis=1)

    return bidirect_hgru_dict

bidirect_hgru_dict=get_df_with_predictions_with_hgru(prediction_dic, hgru_prediction_dic, test_dict)

In [10]:
bidirect_hgru_dict['All items'].head()

,Category,Date,Year,Indent,Actual,Bidirectional_Prediction,HGRU_Prediction
0,All items,2021-01-15,2021,0.0,0.354022,0.391353,0.235949
1,All items,2021-02-15,2021,0.0,0.618238,0.441331,0.344591
2,All items,2021-03-15,2021,0.0,0.767086,0.539811,0.488324
3,All items,2021-04-15,2021,0.0,0.642159,0.645130,0.547848
4,All items,2021-05-15,2021,0.0,0.900787,0.641802,0.526242


In [11]:
len(bidirect_hgru_dict.keys())

349

In [12]:
all_data_test_dict['All items'].head()

,Category,Date,Year,Indent,Actual,Prediction
0,All items,2021-01-15,2021,0.0,0.354022,0.391353
1,All items,2021-02-15,2021,0.0,0.618238,0.441331
2,All items,2021-03-15,2021,0.0,0.767086,0.539811
3,All items,2021-04-15,2021,0.0,0.642159,0.645130
4,All items,2021-05-15,2021,0.0,0.900787,0.641802


In [13]:
all_categories = list(all_data_test_dict.keys())

In [14]:
len(all_categories)

349

In [15]:
def plot_results(all_data_dict, categories):
    #category_samples = random.sample(categories, 20)+['All items']
    category_samples = ['All items'] + categories
    for category in category_samples:
        category_df = all_data_dict[category]
        y_pred = category_df['Prediction'].values
        y_actual = category_df['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)
        print(f'Category is: {category}')
        print(f'RMSE is: {np.sqrt(mse)}')

        fig = px.line(category_df, x="Date", y=["Actual", "Prediction"], title=f'{category} - Actual VS Prediction')
        fig.show()

#plot_results(all_data_test_dict, all_categories)

In [16]:
def plot_results_bidirect_hgru(bidirect_hgru_dict, categories):
    #category_samples = random.sample(categories, 20)+['All items']
    category_samples = ['All items'] + categories
    for category in category_samples:
        category_df = bidirect_hgru_dict[category]
        y_pred_bidirect = category_df['Bidirectional_Prediction'].values
        y_pred_hgru = category_df['HGRU_Prediction'].values
        y_actual = category_df['Actual'].values
        bidirect_mse = mean_squared_error(y_pred_bidirect, y_actual)
        hgru_mse = mean_squared_error(y_pred_hgru, y_actual)
        print(f'Category is: {category}')
        print(f'Bidirectional RMSE is: {np.sqrt(bidirect_mse)}')
        print(f'HGRU RMSE is: {np.sqrt(hgru_mse)}')

        fig = px.line(category_df, x="Date", y=["Actual", "Bidirectional_Prediction", "HGRU_Prediction"], title=f'{category} - Actual VS Prediction')
        fig.show()

#plot_results(all_data_test_dict, all_categories)

In [17]:
def avg_rmse(all_data_test_dict):
    mse_lst = []
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)
        mse_lst.append(mse)
    
    rmse_list = list(map(np.sqrt,mse_lst))
    avg_rmse = np.mean(rmse_list)
    rmse_std = np.std(rmse_list)
    
    print(f'RMSE:  {avg_rmse}')
    print(f'MSE std:  {rmse_std}')
    print(f'interval: {[avg_rmse-rmse_std, avg_rmse+rmse_std]}')

    return avg_rmse,rmse_std

In [18]:
avg_rmse(all_data_test_dict)

RMSE:  1.2494876688810856
MSE std:  1.027822494897479
interval: [0.22166517398360663, 2.277310163778565]


(1.2494876688810856, 1.027822494897479)

In [19]:
def total_corr(all_data_test_dict):
    corr_dict = {}
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        corr = stats.pearsonr(y_pred,y_actual)[0]
        corr_dict[key] =  corr
    
    total_corr = sum(corr_dict.values())
    #avg_corr = total_corr/len(list(all_data_test_dict.keys()))
    
    num_high_corr = 0
    for category in corr_dict:
        if corr_dict[category] >= 0.5:
            num_high_corr +=1
    
    print(f'Number of categories with High Correlation: {num_high_corr}')
    
    return total_corr

In [20]:
total_corr(all_data_test_dict)

Number of categories with High Correlation: 61


102.25600522264224

In [21]:
def avg_r_squared(all_data_test_dict):
    r_squared_lst = []
    low_r_squared_indent = []
    positive_r_squared_indent = []
    high_r_squared_indent = []
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        r2 = r2_score(y_actual, y_pred)
        r_squared_lst.append(r2)

        if r2<=0:
            low_r_squared_indent.append(df_predictions['Indent'].unique().item())
        
        else:
            positive_r_squared_indent.append(df_predictions['Indent'].unique().item())


        if r2>=0.5:
            high_r_squared_indent.append(df_predictions['Indent'].unique().item())

        if key =='All items':
            headline_r2 = r2
    
    avg_r2_score = np.mean(r_squared_lst)
    r2_std = np.std(r_squared_lst)
    
    print(f'Average R Squared:  {avg_r2_score}')
    print(f'R Squared std:  {r2_std}')
    print(f'interval: {[avg_r2_score-r2_std, avg_r2_score+r2_std]}')
    print('--------------------------------------------------------')
    #print(f'sorted r2 list: {sorted(r_squared_lst, reverse=True)}')
    print(f'Headline R2: {headline_r2}')
    print(f'R2 list percentiles:\n[10: {np.percentile(r_squared_lst, 10)}, 25: {np.percentile(r_squared_lst, 25)}, 50: {np.percentile(r_squared_lst, 50)}, 75: {np.percentile(r_squared_lst, 75)}, 90: {np.percentile(r_squared_lst, 90)}]')
    print(f'num of neg/zero R^2: {sum(1 for i in r_squared_lst if i<= 0)}')
    print(f'num of postive R^2: {sum(1 for i in r_squared_lst if i> 0)}')
    print(f'num of high R^2: {sum(1 for i in r_squared_lst if i>= 0.5)}')

    return headline_r2, low_r_squared_indent, positive_r_squared_indent, high_r_squared_indent

In [22]:
headline_r2, low_r_squared_indent_lst, positive_r_squared_indent_lst, high_r_squared_indent_lst = avg_r_squared(all_data_test_dict)

Average R Squared:  -0.031069095880924793
R Squared std:  0.29909191845960215
interval: [-0.3301610143405269, 0.2680228225786774]
--------------------------------------------------------
Headline R2: 0.17724859930400427
R2 list percentiles:
[10: -0.3101866530112369, 25: -0.13918209945622162, 50: 0.0017866788735901862, 75: 0.1208161212825708, 90: 0.2540771193011061]
num of neg/zero R^2: 173
num of postive R^2: 176
num of high R^2: 4


In [23]:
avg_r_squared(all_data_test_dict)

Average R Squared:  -0.031069095880924793
R Squared std:  0.29909191845960215
interval: [-0.3301610143405269, 0.2680228225786774]
--------------------------------------------------------
Headline R2: 0.17724859930400427
R2 list percentiles:
[10: -0.3101866530112369, 25: -0.13918209945622162, 50: 0.0017866788735901862, 75: 0.1208161212825708, 90: 0.2540771193011061]
num of neg/zero R^2: 173
num of postive R^2: 176
num of high R^2: 4


(0.17724859930400427,
 [7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  7.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  6.0,
  3.0,
  3.0,
  3.0,
  3.0,
  3.0,
  3.0,
  3.0,
  3.0,
  3.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  4.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0,
  5.0

In [24]:
high_r_squared_indent_lst

[6.0, 4.0, 4.0, 5.0]

In [25]:
indent_list = [0,1,2,3,4,5,6,7,8]
indent_count = {}

for indent in indent_list:
    indent_count[indent] = [f'low: {low_r_squared_indent_lst.count(indent)}, pos: {positive_r_squared_indent_lst.count(indent)}, high: {high_r_squared_indent_lst.count(indent)}']

#list2.count(element)


In [26]:
indent_count

{0: ['low: 0, pos: 1, high: 0'],
 1: ['low: 13, pos: 12, high: 0'],
 2: ['low: 7, pos: 17, high: 0'],
 3: ['low: 9, pos: 16, high: 0'],
 4: ['low: 21, pos: 27, high: 2'],
 5: ['low: 60, pos: 50, high: 1'],
 6: ['low: 37, pos: 36, high: 1'],
 7: ['low: 24, pos: 12, high: 0'],
 8: ['low: 2, pos: 5, high: 0']}

In [27]:
len(all_categories)

349

In [28]:
def create_rmse_table(all_data_test_dict):
    categories = []
    indents = []
    mses = []
    rmses = []
    for key in all_categories:
        categories.append(key)

        df_predictions = all_data_test_dict[key]
        indents.append(df_predictions.Indent.unique().item())
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)

        mses.append(mse)
        rmses.append(np.sqrt(mse))
    
    series1 = pd.DataFrame(categories).reset_index(drop=True).rename(columns={0:'Category'})
    series2 = pd.DataFrame(indents).reset_index(drop=True).rename(columns={0:'Indent'})
    series3 = pd.DataFrame(mses).reset_index(drop=True).rename(columns={0:'MSE'})
    series4 = pd.DataFrame(rmses).reset_index(drop=True).rename(columns={0:'RMSE'})

    df1 = pd.concat([series1, series2], axis = 1)
    df2 = pd.concat([df1, series3], axis = 1)
    df3 = pd.concat([df2, series4], axis = 1)

    return df3


In [29]:
from pathlib import Path
def rmdir(dirc):
    dirc = Path(dirc)
    for itm in dirc.iterdir():
        if itm.is_dir():
            rmdir(itm)
            print("Deleting", itm, ".... successful.")
        else:
            itm.unlink()
    dirc.rmdir()

In [30]:
path = os.getcwd()

desired_path = path + '/rmse_per_category.csv'

rmse_per_cat = create_rmse_table(all_data_test_dict)
rmse_per_cat.to_csv(desired_path, index=False)

In [31]:
rmse_per_cat.sort_values(by = 'RMSE', ascending=True)

,Category,Indent,MSE,RMSE
84,Owners’ equivalent rent of primary residence,6.0,0.007139,0.084490
172,Rent of shelter,4.0,0.018237,0.135046
292,Rent of primary residence,5.0,0.019741,0.140504
308,"Tuition, other school fees, and childcare",5.0,0.020252,0.142309
331,Education,2.0,0.022197,0.148988
...,...,...,...,...
125,Motor fuel,3.0,26.729268,5.170036
147,Gasoline all types,4.0,27.371760,5.231803
117,Fuel oil and other fuels,3.0,29.625981,5.442975
239,"Gasoline, unleaded regular",5.0,31.486362,5.611271


In [32]:
good_cats = list(rmse_per_cat.sort_values(by = 'RMSE', ascending=True).head(20).Category.unique())

In [33]:
good_cats

['Owners’ equivalent rent of primary residence',
 'Rent of shelter',
 'Rent of primary residence',
 'Tuition, other school fees, and childcare',
 'Education',
 'Shelter',
 'College tuition and fees',
 'Water and sewer and trash collection services',
 'Housing',
 'Education and communication',
 'Services less energy services',
 'Food away from home',
 'Education and communication services',
 'All items less food and energy',
 'Other services',
 'Limited service meals and snacks',
 'Services',
 'Housing at school, excluding board',
 'Food',
 'All items less food, shelter, energy, and used cars and trucks']

In [42]:
categories_with_highest_diff = ["Girls’ apparel",
                                "Vehicle parts and equipment other than tires",
                                "Propane, kerosene, and firewood", 
                                "Purchase of pets, pet supplies, accessories",
                                "Floor coverings",
                                "Personal care",
                                "Fuel oil",
                                "Distilled spirits at home",
                                "Sports equipment",
                                "Other recreation services",
                                "Water and sewerage maintenance",
                                "Motor fuel",
                                "Checking account and other bank services"
                                ]

In [43]:
plot_results(all_data_test_dict, categories_with_highest_diff)

Category is: All items
RMSE is: 0.3129796445419268


Category is: Girls’ apparel
RMSE is: 2.2282906739784325


Category is: Vehicle parts and equipment other than tires
RMSE is: 1.176281669668337


Category is: Propane, kerosene, and firewood
RMSE is: 1.9393277357314527


Category is: Purchase of pets, pet supplies, accessories
RMSE is: 1.100152522627247


...

Category is: Sports equipment
RMSE is: 0.8872362887876798


Category is: Other recreation services
RMSE is: 0.6402262250528237


Category is: Water and sewerage maintenance
RMSE is: 0.26676835939931204


Category is: Motor fuel
RMSE is: 5.1700356311820155


Category is: Checking account and other bank services
RMSE is: 2.4501100110737077


In [44]:
plot_results_bidirect_hgru(bidirect_hgru_dict, categories_with_highest_diff)

Category is: All items
Bidirectional RMSE is: 0.3129796445419268
HGRU RMSE is: 0.3413976060615018


Category is: Girls’ apparel
Bidirectional RMSE is: 2.2282906739784325
HGRU RMSE is: 2.2358370164618413


Category is: Vehicle parts and equipment other than tires
Bidirectional RMSE is: 1.176281669668337
HGRU RMSE is: 4.79697789297253


Category is: Propane, kerosene, and firewood
Bidirectional RMSE is: 1.9393277357314527
HGRU RMSE is: 4.898797066919668


Category is: Purchase of pets, pet supplies, accessories
Bidirectional RMSE is: 1.100152522627247
HGRU RMSE is: 1.1988016723824924


...

Category is: Sports equipment
Bidirectional RMSE is: 0.8872362887876798
HGRU RMSE is: 0.9393519384463661


Category is: Other recreation services
Bidirectional RMSE is: 0.6402262250528237
HGRU RMSE is: 0.7085140376803668


Category is: Water and sewerage maintenance
Bidirectional RMSE is: 0.26676835939931204
HGRU RMSE is: 0.26866035876692734


Category is: Motor fuel
Bidirectional RMSE is: 5.1700356311820155
HGRU RMSE is: 6.772147940249378


Category is: Checking account and other bank services
Bidirectional RMSE is: 2.4501100110737077
HGRU RMSE is: 4.013673712929953
